In [1]:
import pandas as pd
import numpy as np
from functions.tools import *
from numpy.random import seed
seed(30)
#import tensorflow as tf

df = pd.read_pickle(r'./combined_data_feb2023.pkl')

In [2]:
features = ['Ward Glucose', 'Haemoglobin',
       'Mean cell volume, blood', 'White blood cell count, blood',
       'Haematocrit', 'Platelets', 'Urea level, blood', 'Creatinine', 'Sodium',
       'Potassium', 'Lymphocytes', 'Neutrophils', 'C-Reactive Protein',
       'Eosinophils', 'Alkaline Phosphatase', 'Albumin',
       'Alanine Transaminase', 'Bilirubin', 'Total Protein',
       'Fibrinogen (clauss)', 'Glucose POCT Strip Blood', 'Ferritin',
       'D-Dimer', 'Ward Lactate', 'age', 'sex', 'SARS CoV-2 RNA']

In [3]:
#panels for feature selection

full_features = ['Ward Glucose', 'Haemoglobin',
       'Mean cell volume, blood', 'White blood cell count, blood',
       'Haematocrit', 'Platelets', 'Urea level, blood', 'Creatinine', 'Sodium',
       'Potassium', 'Lymphocytes', 'Neutrophils', 'C-Reactive Protein',
       'Eosinophils', 'Alkaline Phosphatase', 'Albumin',
       'Alanine Transaminase', 'Bilirubin', 'Total Protein',
       'Fibrinogen (clauss)', 'Glucose POCT Strip Blood', 'Ferritin',
       'D-Dimer', 'Ward Lactate', 'age', 'sex', 'SARS CoV-2 RNA']

fbc = ['Haemoglobin',
       'Mean cell volume, blood', 
       'White blood cell count, blood',
       'Haematocrit', 
       'Platelets',
       'Lymphocytes', 
       'Neutrophils',
       'Eosinophils']

demographics = ['age','sex']

crp = ['C-Reactive Protein']

ue = ['Urea level, blood', 
       'Creatinine', 
       'Sodium',
       'Potassium']

lft = ['Alkaline Phosphatase', 
       'Albumin',
       'Alanine Transaminase', 
       'Bilirubin', 
       'Total Protein']

clotting = ['Fibrinogen (clauss)',
              'D-Dimer',
              'Ferritin']

lactate = ['Ward Lactate']

covid = ['SARS CoV-2 RNA']

panel1 = demographics+fbc
panel2 = demographics+fbc+ue+lft
panel3 = demographics+fbc+ue+lft+crp
panel4 = demographics+fbc+ue+lft+crp+clotting+lactate
panel5 = full_features


In [4]:
# Train and test set given this undergoes 5-fold cross validation

start_date = '2014-1-1'
val_date = '2021-03-1'

train = df[(df['date_culture']>=start_date)&(df['date_culture']<val_date)]
test = df[df['date_culture']>=val_date]

display('Train')
display(train['pathogenic'].value_counts(normalize=True))
display(len(train.groupby(['id','date_culture'])))

display('Test')
display(test['pathogenic'].value_counts(normalize=True))
display(len(test.groupby(['id','date_culture'])))

'Train'
pathogenic
0.0    0.777018
1.0    0.222982
Name: proportion, dtype: float64

15212
'Test'
pathogenic
0.0    0.915928
1.0    0.084072
Name: proportion, dtype: float64

5638

In [5]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupKFold

In [6]:
split = 5
fraction = (split-1)/split

cv = GroupKFold(n_splits=split)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import LSTM
from keras import regularizers
from keras.layers import BatchNormalization
import tensorflow as tf
from keras.optimizers import Adam

neg, pos = np.bincount(train['pathogenic'])
initial_bias = np.log([pos/neg])
initial_bias

model_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3, restore_best_weights=True)
    ]

METRICS = [

      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.TruePositives(thresholds=None, name=None, dtype=None),
      tf.keras.metrics.TrueNegatives(thresholds=None, name=None, dtype=None),
      tf.keras.metrics.FalsePositives(thresholds=None, name=None, dtype=None),
      tf.keras.metrics.FalseNegatives(thresholds=None, name=None, dtype=None)

    
]

from sklearn.preprocessing import StandardScaler
scale = StandardScaler()



2023-08-15 09:51:22.781680: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-15 09:51:23.923279: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-15 09:51:25.247382: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [7]:

def run_model(train, test, feature):
    summary_val = pd.DataFrame()
    summary_test = pd.DataFrame()
    for train_index, test_index in cv.split(train, train['pathogenic'],  train['id']):
        X_train = train.iloc[train_index]
        X_val = train.iloc[test_index]

        train_samples = len(X_train.groupby('id'))
        train_steps = len(X_train)/ len(X_train.groupby('id'))
        train_steps = int(train_steps)
        train_feature_size = len(X_train[feature].columns)
        train_feature_size = int(train_feature_size)

        val_samples = len(X_val.groupby('id'))
        val_steps = len(X_val)/ len(X_val.groupby('id'))
        val_steps = int(val_steps)
        val_feature_size = len(X_val[feature].columns)
        val_feature_size = int(val_feature_size)

        y_train = X_train.groupby(['id'])['pathogenic'].max().astype(int).values
        y_train = y_train.reshape(len(y_train),1)

        y_val = X_val.groupby(['id'])['pathogenic'].max().astype(int).values
        y_val = y_val.reshape(len(y_val),1)

        X_train_scaled=X_train.copy()
        X_train_scaled[feature] = scale.fit_transform(X_train[feature])
        X_train = X_train_scaled
        
        X_val_scaled=X_val.copy()
        X_val_scaled[feature] = scale.transform(X_val[feature])
        X_val = X_val_scaled

        X_test = test
        X_test[feature] = scale.transform(test[feature])
        
        test_samples = len(test.groupby('id'))
        test_steps = len(test)/ len(test.groupby('id'))
        test_steps = int(test_steps)
        test_feature_size = len(test[feature].columns)

        y_test = test.groupby(['id'])['pathogenic'].max().astype(int).values
        y_test = y_test.reshape(len(y_test),1)

        X_train = X_train[feature].fillna(0).values.reshape(train_samples,train_steps,train_feature_size)
        X_val = X_val[feature].fillna(0).values.reshape(val_samples,val_steps,val_feature_size)
        X_test = X_test[feature].fillna(0).values.reshape(test_samples, test_steps, test_feature_size)

        
        model = Sequential()
        model.add(tf.keras.layers.Masking(mask_value=0., input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(LSTM(10, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        model.add(LSTM(10))
        model.add(Dense(1, activation= 'sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC(curve='ROC',from_logits=True, name='auc'),tf.keras.metrics.AUC(curve='PR',from_logits=True, name='auprc')])

        history = model.fit(X_train,y_train, validation_split=0.2, epochs=20, batch_size=64, callbacks=model_callbacks)

        #Scores on CV for one-fold
        val_outputs = model.evaluate(X_val,y_val, return_dict=True)
        results_val = pd.DataFrame.from_dict(val_outputs,orient='index')
        summary_val = pd.concat([summary_val,results_val],axis=1)

        test_outputs = model.evaluate(X_test,y_test, return_dict=True)
        results_test = pd.DataFrame.from_dict(test_outputs,orient='index')
        summary_test = pd.concat([summary_test, results_test],axis=1)
  
    a=pd.concat([summary_val,summary_test],axis=1)
    a.columns=[
                                                        '1_val',
                                                        '2_val',
                                                        '3_val',
                                                        '4_val',
                                                        '5_val',
                                                        '1_test',
                                                        '2_test',
                                                        '3_test',
                                                        '4_test',
                                                        '5_test']
    return(a)





    


In [8]:
#The full panel
result_panel5 = run_model(train, test, panel5)
display(result_panel5)

/tmp/ipykernel_23391/115137444.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[feature] = scale.transform(test[feature])
2023-08-15 09:51:29.734603: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 15770700 exceeds 10% of free system memory.
/tmp/ipykernel_23391/115137444.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[feature] = scale.transform(test[feature])
2023-08-15 09:52:11.736143: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 15770700 exc

Epoch 1/20
153/153 [==============================] - 15s 42ms/step - loss: 0.4013 - auc: 0.7518 - auprc: 0.3130 - val_loss: 1.0116 - val_auc: 0.6065 - val_auprc: 0.6870
Epoch 2/20
153/153 [==============================] - 3s 22ms/step - loss: 0.2368 - auc: 0.9080 - auprc: 0.6045 - val_loss: 0.8012 - val_auc: 0.6475 - val_auprc: 0.7351
Epoch 3/20
153/153 [==============================] - 3s 21ms/step - loss: 0.1965 - auc: 0.9417 - auprc: 0.6745 - val_loss: 0.7635 - val_auc: 0.6755 - val_auprc: 0.7506
Epoch 4/20
153/153 [==============================] - 3s 20ms/step - loss: 0.1736 - auc: 0.9528 - auprc: 0.7188 - val_loss: 0.7797 - val_auc: 0.6888 - val_auprc: 0.7554
Epoch 5/20
153/153 [==============================] - 3s 22ms/step - loss: 0.1634 - auc: 0.9578 - auprc: 0.7386 - val_loss: 0.7638 - val_auc: 0.6982 - val_auprc: 0.7639

,1_val,2_val,3_val,4_val,5_val,1_test,2_test,3_test,4_test,5_test
loss,0.309193,0.281867,0.288200,0.293654,0.267343,0.159544,0.170141,0.875495,1.181650,0.919525
auc,0.909172,0.917707,0.915472,0.913081,0.934018,0.940046,0.928126,0.578564,0.341369,0.598471
auprc,0.683236,0.726715,0.694590,0.709453,0.803561,0.583835,0.566885,0.204322,0.066048,0.318464


In [9]:
result_panel1 = run_model(train, test, panel1)
display(result_panel1)


/tmp/ipykernel_23391/115137444.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[feature] = scale.transform(test[feature])
/tmp/ipykernel_23391/115137444.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[feature] = scale.transform(test[feature])
/tmp/ipykernel_23391/115137444.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

Epoch 1/20
153/153 [==============================] - 55s 153ms/step - loss: 0.3831 - auc: 0.7575 - auprc: 0.3707 - val_loss: 0.9521 - val_auc: 0.5927 - val_auprc: 0.6791
Epoch 2/20
153/153 [==============================] - 11s 71ms/step - loss: 0.2461 - auc: 0.8971 - auprc: 0.5782 - val_loss: 0.8417 - val_auc: 0.6260 - val_auprc: 0.7050
Epoch 3/20
153/153 [==============================] - 12s 79ms/step - loss: 0.2163 - auc: 0.9265 - auprc: 0.6247 - val_loss: 0.7808 - val_auc: 0.6458 - val_auprc: 0.7196
Epoch 4/20
153/153 [==============================] - 10s 63ms/step - loss: 0.1936 - auc: 0.9435 - auprc: 0.6651 - val_loss: 0.8002 - val_auc: 0.6573 - val_auprc: 0.7337
Epoch 5/20
153/153 [==============================] - 9s 61ms/step - loss: 0.1797 - auc: 0.9501 - auprc: 0.6863 - val_loss: 0.7848 - val_auc: 0.6678 - val_auprc: 0.7420

,1_val,2_val,3_val,4_val,5_val,1_test,2_test,3_test,4_test,5_test
loss,0.335328,0.284383,0.291805,0.302875,0.300675,0.208435,0.162396,0.761203,0.355053,0.997917
auc,0.894436,0.912647,0.907942,0.901593,0.908487,0.883767,0.957087,0.529072,0.844922,0.187124
auprc,0.666045,0.699202,0.674002,0.666675,0.666659,0.463533,0.533859,0.171603,0.433692,0.048072


In [10]:
result_panel2 = run_model(train, test, panel2)
display(result_panel2)
display(result_panel2.T)

/tmp/ipykernel_23391/115137444.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[feature] = scale.transform(test[feature])
/tmp/ipykernel_23391/115137444.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[feature] = scale.transform(test[feature])
/tmp/ipykernel_23391/115137444.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

Epoch 1/20
153/153 [==============================] - 41s 129ms/step - loss: 0.4131 - auc: 0.7173 - auprc: 0.2665 - val_loss: 1.0649 - val_auc: 0.5792 - val_auprc: 0.6601
Epoch 2/20
153/153 [==============================] - 10s 66ms/step - loss: 0.2557 - auc: 0.8864 - auprc: 0.5582 - val_loss: 0.9281 - val_auc: 0.6048 - val_auprc: 0.6883
Epoch 3/20
153/153 [==============================] - 9s 56ms/step - loss: 0.2147 - auc: 0.9282 - auprc: 0.6244 - val_loss: 0.7883 - val_auc: 0.6358 - val_auprc: 0.7055
Epoch 4/20
153/153 [==============================] - 9s 57ms/step - loss: 0.1893 - auc: 0.9439 - auprc: 0.6689 - val_loss: 0.7919 - val_auc: 0.6471 - val_auprc: 0.7133
Epoch 5/20
153/153 [==============================] - 9s 57ms/step - loss: 0.1728 - auc: 0.9540 - auprc: 0.6859 - val_loss: 0.8103 - val_auc: 0.6601 - val_auprc: 0.7238

,1_val,2_val,3_val,4_val,5_val,1_test,2_test,3_test,4_test,5_test
loss,0.320842,0.290079,0.297381,0.302538,0.335394,0.392521,0.611584,0.631494,0.207968,0.614881
auc,0.903039,0.911730,0.899401,0.909196,0.893629,0.864222,0.573315,0.922631,0.885940,0.936221
auprc,0.662454,0.688245,0.645813,0.707859,0.649863,0.475889,0.165791,0.501900,0.503241,0.546559


,loss,auc,auprc
1_val,0.320842,0.903039,0.662454
2_val,0.290079,0.911730,0.688245
3_val,0.297381,0.899401,0.645813
4_val,0.302538,0.909196,0.707859
5_val,0.335394,0.893629,0.649863
1_test,0.392521,0.864222,0.475889
2_test,0.611584,0.573315,0.165791
3_test,0.631494,0.922631,0.501900
4_test,0.207968,0.885940,0.503241
5_test,0.614881,0.936221,0.546559


In [11]:
result_panel3 = run_model(train, test, panel3)
display(result_panel3)

/tmp/ipykernel_23391/115137444.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[feature] = scale.transform(test[feature])
/tmp/ipykernel_23391/115137444.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[feature] = scale.transform(test[feature])
/tmp/ipykernel_23391/115137444.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

Epoch 1/20
153/153 [==============================] - 28s 87ms/step - loss: 0.4239 - auc: 0.7099 - auprc: 0.3052 - val_loss: 0.9501 - val_auc: 0.5660 - val_auprc: 0.6673
Epoch 2/20
153/153 [==============================] - 6s 41ms/step - loss: 0.2724 - auc: 0.8636 - auprc: 0.5658 - val_loss: 0.9587 - val_auc: 0.6063 - val_auprc: 0.6908
Epoch 3/20
153/153 [==============================] - 7s 45ms/step - loss: 0.2299 - auc: 0.9064 - auprc: 0.6341 - val_loss: 0.8404 - val_auc: 0.6404 - val_auprc: 0.7204
Epoch 4/20
153/153 [==============================] - 8s 53ms/step - loss: 0.2024 - auc: 0.9335 - auprc: 0.7017 - val_loss: 0.7738 - val_auc: 0.6665 - val_auprc: 0.7397
Epoch 5/20
153/153 [==============================] - 9s 61ms/step - loss: 0.1784 - auc: 0.9526 - auprc: 0.7419 - val_loss: 0.7524 - val_auc: 0.6884 - val_auprc: 0.7616

,1_val,2_val,3_val,4_val,5_val,1_test,2_test,3_test,4_test,5_test
loss,0.287916,0.284811,0.270158,0.294598,0.276488,0.252639,0.966571,0.837954,0.762001,0.651158
auc,0.920936,0.913197,0.925475,0.908915,0.922462,0.853144,0.514118,0.683325,0.771117,0.803629
auprc,0.751230,0.685546,0.747392,0.697409,0.746807,0.443503,0.127171,0.366314,0.414565,0.422616


In [12]:
result_panel4 = run_model(train, test, panel4)
display(result_panel4)

/tmp/ipykernel_23391/115137444.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[feature] = scale.transform(test[feature])
/tmp/ipykernel_23391/115137444.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[feature] = scale.transform(test[feature])
/tmp/ipykernel_23391/115137444.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

Epoch 1/20
153/153 [==============================] - 44s 129ms/step - loss: 0.4006 - auc: 0.7356 - auprc: 0.2777 - val_loss: 1.0571 - val_auc: 0.5910 - val_auprc: 0.6750
Epoch 2/20
153/153 [==============================] - 6s 42ms/step - loss: 0.2442 - auc: 0.9014 - auprc: 0.5907 - val_loss: 0.8875 - val_auc: 0.6123 - val_auprc: 0.6889
Epoch 3/20
153/153 [==============================] - 6s 39ms/step - loss: 0.2061 - auc: 0.9357 - auprc: 0.6412 - val_loss: 0.8052 - val_auc: 0.6310 - val_auprc: 0.6943
Epoch 4/20
153/153 [==============================] - 6s 41ms/step - loss: 0.1764 - auc: 0.9522 - auprc: 0.6979 - val_loss: 0.7728 - val_auc: 0.6517 - val_auprc: 0.7068
Epoch 5/20
153/153 [==============================] - 6s 40ms/step - loss: 0.1675 - auc: 0.9554 - auprc: 0.6974 - val_loss: 0.7876 - val_auc: 0.6661 - val_auprc: 0.7240

,1_val,2_val,3_val,4_val,5_val,1_test,2_test,3_test,4_test,5_test
loss,0.286725,0.291844,0.272901,0.277092,0.282188,0.563532,0.596677,0.652829,1.329476,0.596308
auc,0.914716,0.911633,0.925003,0.925355,0.922429,0.859104,0.628508,0.618249,0.270878,0.746611
auprc,0.698118,0.713973,0.739954,0.752736,0.741788,0.455434,0.325994,0.343124,0.069699,0.351151


In [13]:
print('AUC for CV with validation')
display(result_panel1.T.iloc[:5,:]['auc'].describe())
display(result_panel2.T.iloc[:5,:]['auc'].describe())
display(result_panel3.T.iloc[:5,:]['auc'].describe())
display(result_panel4.T.iloc[:5,:]['auc'].describe())
display(result_panel5.T.iloc[:5,:]['auc'].describe())
print()
print('AUPRC fr CV with validation')
display(result_panel1.T.iloc[:5,:]['auprc'].describe())
display(result_panel2.T.iloc[:5,:]['auprc'].describe())
display(result_panel3.T.iloc[:5,:]['auprc'].describe())
display(result_panel4.T.iloc[:5,:]['auprc'].describe())
display(result_panel5.T.iloc[:5,:]['auprc'].describe())
print()
print('AUC for hold out set evaluation')
display(result_panel1.T.iloc[5:,:]['auc'].describe())
display(result_panel2.T.iloc[5:,:]['auc'].describe())
display(result_panel3.T.iloc[5:,:]['auc'].describe())
display(result_panel4.T.iloc[5:,:]['auc'].describe())
display(result_panel5.T.iloc[5:,:]['auc'].describe())
print()
print('AUPRC for hold out set evaluation')
display(result_panel1.T.iloc[5:,:]['auprc'].describe())
display(result_panel2.T.iloc[5:,:]['auprc'].describe())
display(result_panel3.T.iloc[5:,:]['auprc'].describe())
display(result_panel4.T.iloc[5:,:]['auprc'].describe())
display(result_panel5.T.iloc[5:,:]['auprc'].describe())

AUC for CV with validation


count    5.000000
mean     0.905021
std      0.007115
min      0.894436
25%      0.901593
50%      0.907942
75%      0.908487
max      0.912647
Name: auc, dtype: float64
count    5.000000
mean     0.903399
std      0.007324
min      0.893629
25%      0.899401
50%      0.903039
75%      0.909196
max      0.911730
Name: auc, dtype: float64
count    5.000000
mean     0.918197
std      0.006889
min      0.908915
25%      0.913197
50%      0.920936
75%      0.922462
max      0.925475
Name: auc, dtype: float64
count    5.000000
mean     0.919827
std      0.006273
min      0.911633
25%      0.914716
50%      0.922429
75%      0.925003
max      0.925355
Name: auc, dtype: float64
count    5.000000
mean     0.917890
std      0.009554
min      0.909172
25%      0.913081
50%      0.915472
75%      0.917707
max      0.934018
Name: auc, dtype: float64


AUPRC fr CV with validation


count    5.000000
mean     0.674517
std      0.014183
min      0.666045
25%      0.666659
50%      0.666675
75%      0.674002
max      0.699202
Name: auprc, dtype: float64
count    5.000000
mean     0.670847
std      0.026505
min      0.645813
25%      0.649863
50%      0.662454
75%      0.688245
max      0.707859
Name: auprc, dtype: float64
count    5.000000
mean     0.725677
std      0.031546
min      0.685546
25%      0.697409
50%      0.746807
75%      0.747392
max      0.751230
Name: auprc, dtype: float64
count    5.000000
mean     0.729314
std      0.022505
min      0.698118
25%      0.713973
50%      0.739954
75%      0.741788
max      0.752736
Name: auprc, dtype: float64
count    5.000000
mean     0.723511
std      0.047630
min      0.683236
25%      0.694590
50%      0.709453
75%      0.726715
max      0.803561
Name: auprc, dtype: float64


AUC for hold out set evaluation


count    5.000000
mean     0.680394
std      0.320625
min      0.187124
25%      0.529072
50%      0.844922
75%      0.883767
max      0.957087
Name: auc, dtype: float64
count    5.000000
mean     0.836466
std      0.149869
min      0.573315
25%      0.864222
50%      0.885940
75%      0.922631
max      0.936221
Name: auc, dtype: float64
count    5.000000
mean     0.725067
std      0.133171
min      0.514118
25%      0.683325
50%      0.771117
75%      0.803629
max      0.853144
Name: auc, dtype: float64
count    5.000000
mean     0.624670
std      0.220824
min      0.270878
25%      0.618249
50%      0.628508
75%      0.746611
max      0.859104
Name: auc, dtype: float64

count    5.000000
mean     0.677315
std      0.255324
min      0.341369
25%      0.578564
50%      0.598471
75%      0.928126
max      0.940046
Name: auc, dtype: float64


AUPRC for hold out set evaluation


count    5.000000
mean     0.330152
std      0.208994
min      0.048072
25%      0.171603
50%      0.433692
75%      0.463533
max      0.533859
Name: auprc, dtype: float64
count    5.000000
mean     0.438676
std      0.154641
min      0.165791
25%      0.475889
50%      0.501900
75%      0.503241
max      0.546559
Name: auprc, dtype: float64
count    5.000000
mean     0.354834
std      0.130371
min      0.127171
25%      0.366314
50%      0.414565
75%      0.422616
max      0.443503
Name: auprc, dtype: float64
count    5.000000
mean     0.309080
std      0.143124
min      0.069699
25%      0.325994
50%      0.343124
75%      0.351151
max      0.455434
Name: auprc, dtype: float64
count    5.000000
mean     0.347911
std      0.226131
min      0.066048
25%      0.204322
50%      0.318464
75%      0.566885
max      0.583835
Name: auprc, dtype: float64